In [53]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [54]:
def date_change(date):
    nowdate = datetime.strptime(date, "%Y-%m-%d")

    nextYear = nowdate.replace(year=nowdate.year + 1)
    endDate = (nextYear - timedelta(days=1)).strftime("%Y%m%d")
    startDate = nowdate.strftime("%Y%m%d")

    return startDate, endDate

In [55]:
date_change("2024-05-22")

('20240522', '20250521')

In [56]:
def parse_month(year, month, stnid, stnnm, m_temp, m_gtemp):

    try:

        # 지점 이름, 지점 아이디

        return {
            "년도": year,
            "월": month,
            "지점 번호": stnid,
            "지점명": stnnm,
            "월평균 기온": m_temp,
            "월평균 지면온도": m_gtemp,
        }

    except AttributeError as e:
        return {
            "년도": None,
            "월": None,
            "지점 번호": None,
            "지점명": None,
            "월평균 기온": None,
            "월평균 지면온도": None,
        }

In [57]:
def mean_cul(item, result_m):

    STNID = item.find("stnId").get_text()
    STNNM = item.find("stnNm").get_text()
    try:
        temp = float(item.find("avgTa").get_text())
        ground_temp = float(item.find("avgTs").get_text())
    except ValueError:
        temp = 0
        ground_temp = 0
        if "error" in result_m:
            result_m["error"] = result_m["error"] + 1
        else:
            result_m["error"] = 1

    result_m["stnid"] = STNID
    result_m["stnnm"] = STNNM

    if "t_total" in result_m:
        result_m["t_total"] = result_m["t_total"] + temp
        result_m["gt_total"] = result_m["gt_total"] + ground_temp
        result_m["cnt"] = result_m["cnt"] + 1
    else:
        result_m["t_total"] = temp
        result_m["gt_total"] = ground_temp
        result_m["cnt"] = 1

    return result_m

In [58]:
def init(result_m):
    result_m = {}
    return result_m


In [59]:
def data_calling(day, id, month_result):
    startDt, endDt = date_change(day)

    url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"
    params = {
        "serviceKey": "JblClLk+uLRS6hB9k3LbHV8cJYUxFo3XL4pNRgX5yJOdGf8E8wlFkrB8so5AunCF9QnI/tIdf/vkDbpqcWwzKg==",
        "pageNo": 1,
        "numOfRows": "365",
        "dataType": "XML",
        "dataCd": "ASOS",
        "dateCd": "DAY",
        "startDt": startDt,
        "endDt": endDt,
        "stnIds": id,
    }

    response = requests.get(url, params=params)

    # pprint.pprint(response.text, depth=4)

    soup = BeautifulSoup(response.content, features="xml")
    # pprint.pprint(soup.text)
    items = soup.find_all("item")

    if items is None:
        print("nothing in items")

    month = "01"

    result_m = {}
    # result_m = init(result_m)

    for item in items:
        TM = item.find("tm").get_text()

        months = TM.split("-")

        if month == months[1]:
            result_m = mean_cul(item, result_m)

        else:
            if "error" in result_m:
                count = result_m["cnt"] - result_m["error"]
            else:
                count = result_m["cnt"]

            m_temp = result_m["t_total"] / count
            m_gtemp = result_m["gt_total"] / count
            month_result.append(
                parse_month(
                    months[0],
                    month,
                    result_m["stnid"],
                    result_m["stnnm"],
                    m_temp,
                    m_gtemp,
                )
            )
            month = months[1]
            result_m = {}
            # result_m = init(result_m)
            result_m = mean_cul(item, result_m)

    if "error" in result_m:
        count = result_m["cnt"] - result_m["error"]
    else:
        count = result_m["cnt"]

    m_temp = result_m["t_total"] / count
    m_gtemp = result_m["gt_total"] / count

    month_result.append(
        parse_month(
            months[0], month, result_m["stnid"], result_m["stnnm"], m_temp, m_gtemp
        )
    )

    return month_result

In [60]:
import pandas as pd

data = pd.read_csv('./observation_point_info.csv',encoding="CP949")
OPI_df = pd.DataFrame(data)

In [61]:
observation_point_num_list = [90,93,95,98,99,100,101,102,104,105,106,108,112,114,115,119,121,127,129,130,131,133,135,136,137,138,140,143,146,152,155,156,159,162,165,168,169,170,172,174,177,184,185,188,189,192,201,202,203,211,212,216,217,221,226,232,235,236,238,239,243,244,245,247,248,251,252,253,254,255,257,258,259,260,261,262,263,264,266,268,271,272,273,276,277,278,279,281,283,284,285,288,289,294,295]

In [62]:
opi2_df = OPI_df[OPI_df['위도']<37]

In [73]:
len(opi2_df)

110

In [63]:
final_opi_df = opi2_df[opi2_df['위도']>36]

In [64]:
len(final_opi_df)

34

In [65]:
opi_point_list = list(final_opi_df['지점'])

In [66]:
len(opi_point_list)

34

In [69]:
opi_final_list = []
for point in opi_point_list:
    if point in observation_point_num_list:
        opi_final_list.append(point)

In [74]:
len(opi_final_list)

32

In [70]:
month_result = []
date_list = [
    "2014-01-01",
    "2015-01-01",
    "2016-01-01",
    "2017-01-01",
    "2018-01-01",
    "2019-01-01",
    "2020-01-01",
    "2021-01-01",
    "2022-01-01",
    "2023-01-01",
]
# id_list = ["127"]
id_list = opi_final_list

In [71]:
for date in date_list:
    for id in id_list:
        month_result = data_calling(date, id, month_result)


df = pd.DataFrame(month_result)

KeyError: 'cnt'